In [ ]:
!git clone https://github.com/nikitalokhmachev-ai/huggingface-semantic-interior-segmentation


Cloning into 'huggingface-semantic-interior-segmentation'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 32 (delta 9), reused 28 (delta 7), pack-reused 0
Receiving objects: 100% (32/32), 505.66 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s e

In [ ]:
import sys
sys.path.append('/content/huggingface-semantic-interior-segmentation')


In [ ]:
import gradio as gr
from gradio.components import Image as ImageGR
import glob
import torch
from PIL import Image, ImageDraw
import numpy as np
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
from class_info import class_names, class_colors, class_ids
import numpy as np
from scipy.ndimage import center_of_mass


def combine_ims(im1, im2, val=128):
  p = Image.new("L", im1.size, val)
  im = Image.composite(im1, im2, p)
  return im

def get_class_centers(segmentation_mask, class_dict):
    segmentation_mask = segmentation_mask.numpy() + 1
    class_centers = {}
    for class_index, _ in class_dict.items():
        class_mask = (segmentation_mask == class_index).astype(int)
        center_of_mass_list = center_of_mass(class_mask)

        class_centers[class_index] = center_of_mass_list

    class_centers = {k:list(map(int, v)) for k,v in class_centers.items() if not np.isnan(sum(v))}
    return class_centers

def visualize_mask(predicted_semantic_map, class_ids, class_colors):
  h, w = predicted_semantic_map.shape
  color_indexes = np.zeros((h, w), dtype=np.uint8)
  color_indexes[:] = predicted_semantic_map.numpy()
  color_indexes = color_indexes.flatten()

  colors = class_colors[class_ids[color_indexes]]
  output = colors.reshape(h, w, 3).astype(np.uint8)
  image_mask = Image.fromarray(output)
  return image_mask


def get_out_image(image, predicted_semantic_map):
  class_centers = get_class_centers(predicted_semantic_map, class_dict)
  mask = visualize_mask(predicted_semantic_map, class_ids, class_colors)
  image_mask = combine_ims(image, mask, val=128)
  draw = ImageDraw.Draw(image_mask)
  for id, (y, x) in class_centers.items():
    draw.text((x, y), str(class_names[id-1]), fill='black')

  return image_mask

def gradio_process(image):
  inputs = processor(images=image, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs)

  predicted_semantic_map = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]

  out_image = get_out_image(image, predicted_semantic_map)
  return out_image

#class_names, class_ids, class_colors = joblib.load('ade20k_classes.joblib')
class_names, class_ids, class_colors = np.array(class_names), np.array(class_ids), np.array(class_colors)
class_dict = dict(zip(class_ids, class_names))

device = torch.device("cpu")
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-ade-semantic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-ade-semantic").to(device)
model.eval()

demo = gr.Interface(
    gradio_process,
    inputs=ImageGR(type="pil"),
    outputs=ImageGR(type="pil"),
    title="Semantic Interior Segmentation",
    examples=glob.glob('./examples/*.jpg'),
    allow_flagging="never"
)

demo.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/82.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/866M [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c17a388ce52b881cf6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
